Filtering to get only main diagnosis... should this be done? Why aren't the other diagnoses important?

Instead we could get the encounter/main diagnoses in another table, then join on the entire diagnoses table to keep the other diagnoses

In [31]:
import pickle
import time

import numpy as np
from querying import (
    get_bt_for_cohort,
    get_cohort,
    get_er_for_cohort,
    get_labs,
    get_most_recent_encounters,
    get_non_cardiac_diagnoses,
    main,
)
from consts import OUTCOME_DEATH, OUTCOME_EDEMA

from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    CARE_UNIT,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    ENCOUNTER_ID,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    HOSPITAL_ID,
    SEX,
    SUBJECT_ID,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STRING,
    TARGETS,
)
from cyclops.utils.file import load_dataframe, save_dataframe

# Querying

In [3]:
t = time.time()
cohort, labs = main()
print(time.time() - t)
cohort

2022-07-19 17:24:09,695 INFO cyclops.orm     - Query returned successfully!
2022-07-19 17:24:09,699 INFO cyclops.utils.profile - Finished executing function run_query in 3.886949 s
2022-07-19 17:24:11,253 INFO cyclops.orm     - Query returned successfully!
2022-07-19 17:24:11,256 INFO cyclops.utils.profile - Finished executing function run_query in 1.555096 s
2022-07-19 17:24:22,335 INFO cyclops.orm     - Query returned successfully!
2022-07-19 17:24:22,339 INFO cyclops.utils.profile - Finished executing function run_query in 10.737625 s
2022-07-19 17:24:27,167 INFO cyclops.orm     - Query returned successfully!
2022-07-19 17:24:27,170 INFO cyclops.utils.profile - Finished executing function run_query in 1.139400 s
2022-07-19 17:24:27,718 INFO cyclops.orm     - Query returned successfully!
2022-07-19 17:24:27,721 INFO cyclops.utils.profile - Finished executing function run_query in 0.232176 s
2022-07-19 17:25:27,267 INFO cyclops.orm     - Query returned successfully!
2022-07-19 17:25:2

449.71076798439026


,encounter_id,admit_timestamp,discharge_timestamp,age,sex,hospital_id,outcome_death,readmission,from_nursing_home_mapped,from_acute_care_institution_mapped,...,X-ray_shoulder,CT_shoulder,MRI_shoulder,Ultrasound_shoulder,X-ray_whole_body,CT_whole_body,MRI_whole_body,Ultrasound_whole_body,ip_charlson_derived,outcome_edema
0,15941876,2015-04-23 20:00:00,2015-05-11 13:22:00,23,M,THPC,False,new_to_acute,False,False,...,0,0,0,0,0,0,0,0,2,False
1,12182859,2010-11-22 01:00:00,2010-11-22 15:00:00,29,M,SBK,False,new_to_acute,False,False,...,0,0,0,0,0,0,0,0,0,False
2,15746021,2016-11-02 16:40:00,2016-11-14 09:38:00,25,F,THPC,False,new_to_acute,False,False,...,0,0,0,0,0,0,0,0,0,False
3,15834641,2014-03-03 21:30:00,2014-03-07 22:15:00,81,M,THPC,False,new_to_acute,False,False,...,0,0,0,0,0,0,0,0,0,False
4,15274625,2015-09-16 02:50:00,2015-10-01 14:20:00,49,F,THPC,False,new_to_acute,False,False,...,0,0,0,0,0,0,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108081,14167874,2020-01-20 19:55:00,2020-01-31 14:43:00,54,M,MSH,True,nota,False,False,...,0,0,0,0,0,0,0,0,6,False
108082,13933236,2018-12-29 20:53:00,2019-01-21 13:00:00,57,M,UHNTG,True,nota,False,False,...,0,0,0,0,2,0,1,0,6,False
108083,15508246,2018-04-05 18:14:00,2018-04-17 18:03:00,91,M,THPM,True,nota,False,False,...,0,0,0,0,0,0,0,0,2,False
108084,15647602,2019-12-23 22:35:00,2020-01-24 11:30:00,82,M,THPC,True,nota,False,False,...,1,0,0,0,0,0,0,0,2,False


In [4]:
save_dataframe(cohort, "cohort_raw.parquet")
save_dataframe(labs, "labs_raw.parquet")

2022-07-19 17:31:35,565 INFO cyclops.utils.file - Saving dataframe to cohort_raw.parquet
2022-07-19 17:31:36,047 INFO cyclops.utils.file - Saving dataframe to labs_raw.parquet


'labs_raw.parquet'

In [5]:
cohort[OUTCOME_DEATH].sum() / len(cohort)

0.10348241215328535

In [6]:
cohort[OUTCOME_EDEMA].sum() / len(cohort)

0.0384416113095128

In [ ]:
# Intersect over encounter IDs to get only those encounters common to both
cohort, labs = intersect_datasets([cohort, labs], ENCOUNTER_ID)

In [13]:
save_dataframe(cohort, "cohort.parquet")
save_dataframe(labs, "labs.parquet")

2022-07-19 17:33:20,114 INFO cyclops.utils.file - Saving dataframe to cohort.parquet
2022-07-19 17:33:20,687 INFO cyclops.utils.file - Saving dataframe to labs.parquet


'labs.parquet'